# GIỚI THIỆU BÀI TOÁN


Phân loại các câu hỏi trên Quora xem chúng là thiếu chân thành hay không.


Em đã dùng một số mô hình như là Logistic Regression hay SVM để giải quyết bài toán nhưng trong bài toán, kaggle có cung cấp một số công cụ Embeddings. Do đó em quyết định tập trung vào các tool Embeddings và so sánh cũng như nhận xét sự khác biệt giữa chúng.

# PHÂN TÍCH DỮ LIỆU

Trước tiên chúng ta import một số thư viện cần thiết

In [ ]:
# Import các thư viện cần thiết
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, LSTM, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Ta sẽ xem xét một chút về các tệp input được cung cấp

In [ ]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)
train_df.head()

Tệp input được cho là 2 tệp train và test. Tệp train gồm có 3 cột lần lượt là mã câu hỏi(qid), nội dung câu hỏi(question_text) và target(= 0 nếu như câu hỏi bình thường, = 1 nếu câu hỏi không chân thành). Có tổng cộng 1306122 câu hỏi trong tệp train và 375806 câu hỏi trong tập test. Dễ dàng thấy được dữ liệu dùng để train chỉ cần hai phần chính: question_text và target.

In [ ]:
test_df.head()

Tệp test khá giống tệp train nhưng không có phần target. Đây là những câu hỏi mà ta cần gán nhãn target(0,1) cũng chính là nhiệm vụ của ta cần giải quyết trong bài toán này.

Xem xét một chút về tính cân bằng của dữ liệu

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
target = train_df['target']
target_1 = 0
for target_value in target:
    if target_value == 1:
        target_1 += 1
print("Số câu hỏi trong tệp train:", len(target))
print("Số câu hỏi được gán nhãn là 1:", target_1)
myLabels = ["insincere question", "sincere question"]
myCounts = [target_1, len(target) - target_1]
plt.pie(myCounts, labels = myLabels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')
plt.show()

Chỉ có 6.2% số câu hỏi trong tập train được gán nhãn là không chân thành (output là 1). Dữ liệu cực kỳ mất cân bằng. Có cách nào để giảm thiểu điều này không? Em nghĩ chúng ta có thể bỏ bớt dữ liệu có nhãn bằng 0 đi. Thêm một số từ không có nghĩa hoặc không ảnh hưởng đến các key-word làm cho câu hỏi trở nên không chân thành, từ đó có thể tạo ra thêm nhiều các câu hỏi gán nhãn 1. Với những dữ liệu mất cân bằng như thế này, chúng ta thường quan tâm nhiều hơn đến việc đánh giá một mô hình bằng F1-score hơn là accuracy

Đây là một bài toán phân loại nhị phân. Một số mô hình ta có thể nghĩ đến là Logistic Regression, SVM, .. nhưng em nghĩ đây là một bài toán về chuỗi nên việc xử lý chuỗi như thế nào trước khi huấn luyện quan trọng hơn là sử dụng mô hình nào. Để kết hợp với Embeddings, em sử dụng GRU model

# HUẤN LUYỆN DEEP LEARNING VỚI MÔ HÌNH GRU

Trước tiên, ta xử lý một chút dữ liệu dùng để train

Chia tệp train thành 2 phần: train và validate. Tệp train sẽ dùng để huấn luyện còn tệp validate sẽ dùng để kiểm tra xem mô hình có tốt không. Điền "na" vào các dữ liệu còn thiếu tránh sự mất mát 

In [ ]:
# lay ra 10% train de lam validate
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2021)

# some config values 
embed_size = 300 # Độ dài của mỗi vector từ
max_features = 50000 # Số lượng từ tối đa trong từ điển sẽ sử dụng
maxlen = 100 # Số lượng từ tối đa trong một câu

# Điền "na" vào các dữ liệu còn trống 
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

Nếu để dữ liệu là các chuỗi thì máy sẽ không hiểu được. Ta sẽ nghĩ đến việc mã hóa mỗi từ thành một số nguyên dương duy nhất. Giả sử một câu có 15 từ thì sẽ được mã hóa là một vector số 15x1. Ta dùng Tokenizer để làm việc này. Nó sẽ mã hóa các từ thành các số nguyên dương duy nhất. Số càng thấp nghĩa là từ đó càng phổ biến trong từ điển. Để đồng nhất dữ liệu ta cũng dùng pad_sequences đảm bảo các câu đều dài 100 từ (Cắt bớt các câu dài hơn 100 từ, điền 0(không đại diện cho từ nào cả) vào cho đủ các câu chưa đủ 100 từ). Khi đó mỗi câu sẽ được đại diện bằng một vector số 100x1

In [ ]:
# mã hóa từ thành số, câu thành vector số 
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

# Đảm bảo mỗi câu hỏi luôn dài 100 từ
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

Ta được tập train_X, val_X, test_X là các vector số tương ứng với mỗi câu hỏi trong các tệp. Xem thử câu hỏi mã số 0 sau khi mã hóa thành như thế nào

In [ ]:
print(train_X[0])

Lấy cột target của tệp train và tệp val để đem đi huấn luyện

In [ ]:
## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

Xây dựng GRU model

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Input sẽ là các vector chuỗi tương ứng với các câu hỏi. Một chuỗi sẽ có 100 từ tương ứng với vector 100 chiều. Tầng Embedding sẽ giúp máy học được nghĩa của các từ là gì. Embedding sẽ chuyển mỗi từ thành 1 vector 1x300 thể hiện nghĩa của từ đấy. Nghĩa là một câu sẽ là một vector số 100x300. Tầng Bidirection sẽ giúp máy học được nghĩa của mỗi câu dựa trên thứ tự của các từ trên mạng noron. Sau đó với mỗi đặc trưng trong 128 đặc trưng, tầng global sẽ chọn ra từ có đặc trưng đó tốt nhất. Các tầng còn lại trong mô hình dùng để phân loại.

Bắt đầu train với tệp train_X train_Y dùng để train. Đưa dữ liệu vào mạng neural network 2 lần. Mỗi lần đưa chia nhỏ ra thành các batch_size là 512 câu. Dữ liệu dùng để kiểm thử là val_X và val_y.

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Mô hình có vẻ hoạt động khá tốt. Để xem xét một cách chi tiết hơn, tính F1-score của mô hình với các threshold từ 0.1 đến 0.5 

In [ ]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))))
    

Ta thấy F1-score trong khoảng threshold 0.26-0.4 là khá tốt. Em thấy điều này khá hợp lý vì ta cần quan tâm hơn đến việc "bỏ sót còn hơn nhầm" nên ta sẽ nên xét threshold < 0.5(gần 0 hơn là 1). Nghĩa là ta thà lỡ bỏ qua việc xác định các câu hỏi thiếu chân thành còn hơn là xác định nhầm một câu hỏi thiếu chân thành là chân thành.

In [ ]:
del model, inp, x
import gc; gc.collect()
time.sleep(10)

Như chúng ta thấy, với mô hình như trên, ở tầng Embedding máy phải tự học nghĩa của các từ. Ta có thể thay đổi hiệu suất mô hình bằng cách sử dụng một số tool Embedding được kaggle cung cấp. Chúng sẽ mã hóa mỗi từ thành một vector số 300 chiều dựa vào ngữ nghĩa của chúng. Các từ giống nhau thì có các vector nhúng tương tự nhau. Thay vì để cho máy phải học, ta sử dụng các tool Embedding được cung cấp từ trước để máy có thể hiểu được nghĩa của các từ. 

# CẢI THIỆN HIỆU SUẤT MÔ HÌNH VỚI MỘT SỐ TOOL EMBEDDING

Xem xét các file nhúng được cung cấp. Chúng được để trong một file zip nên ta tiến hành unzip

In [ ]:
from zipfile import ZipFile
file_name = "../input/quora-insincere-questions-classification/embeddings.zip"
with ZipFile(file_name, 'r') as zip:
     # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Lưu lại đường dẫn đến các file nhúng

In [ ]:
glove = '../working/glove.840B.300d/glove.840B.300d.txt'
paragram =  '../working/paragram_300_sl999/paragram_300_sl999.txt'
wiki_news = '../working/wiki-news-300d-1M/wiki-news-300d-1M.vec'

cài đặt hàm load_embed dùng để load 3 loại nhúng 

In [ ]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == wiki_news:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

Ta sẽ load thử file nhúng Glove Embeddings

In [ ]:
embed_glove = load_embed(glove)

Sau khi load ta được embed_glove. embed_glove sẽ cung cấp cho chúng ta vector số ứng với mỗi từ. Ví dụ đây là vector số của từ "the"

In [ ]:
print(embed_glove['the'])

Nhúng glove embeddings vào các từ đã được mã hóa thành số từ trước bằng tokenizer lưu trong word_index. Thay vì lưu vector số tương ứng với các từ, ta lưu vector số ứng với số đã được mã hóa của từ đấy. Ở đây, em tự đặt 1 ra câu hỏi: File nhúng có đảm bảo mã hóa được tất cả các từ có trong mọi câu hỏi không ? Câu trả lời là không. Vậy ta có một giải pháp: với các từ không được nhúng, ta sẽ lấy phân phối chuẩn của vector số các từ đã biết. 

In [ ]:
# Lấy phân phối chuẩn của các từ đã biết ghi vào mọi từ 
all_embs = np.stack(embed_glove.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

# Replace nếu như từ đó được định nghĩa trong file nhúng
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embed_glove.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
print(word_index['the'])
print(embed_glove['the'])
print(embedding_matrix[2])

2 cách thể hiện vector số của từ "the"

Xây dựng lại model với glove embeddings. Tầng Embedding được thêm weights là ma trận vector số của các từ embedding_matrix


In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# add weights = [embedding_matrix]
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Bắt đầu train với GRU model kèm file nhúng Glove Embeddings

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Có vẻ tốt hơn so với việc ta không dùng Embeddings. Xem xét F1-score

In [ ]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

Kết quả rõ ràng tốt hơn so với việc ta không dùng Embeddings. Tiếp tục thử với Wiki News FastText Embeddings


In [ ]:
# Lưu lại predict
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
del word_index, embed_glove, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

Load wiki news fasttext embeddings

In [ ]:
embed_wiki_news = load_embed(wiki_news)

In [ ]:
all_embs = np.stack(embed_wiki_news.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embed_wiki_news.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# add weights = [embedding_matrix]
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_wiki_news_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

Kết quả tốt hơn so với việc không dùng nhúng, xấp xỉ so với việc dùng Glove Embeddings

In [ ]:
# Lưu lại predict
pred_wiki_news_test_y = model.predict([test_X], batch_size=1024, verbose=1)


In [ ]:
del embed_wiki_news, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

Thử với Paragram Embeddings

In [ ]:
embed_paragram = load_embed(paragram)

In [ ]:
all_embs = np.stack(embed_paragram.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embed_paragram.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# add weights = [embedding_matrix]
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

In [ ]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

Kết quả vẫn xấp xỉ với 2 tool nhúng trước. Ta thử kết hợp cả 3 tool nhúng để xem kết quả có cải thiện hay không

In [ ]:
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_wiki_news_val_y + 0.34*pred_paragram_val_y 
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))

Tốt hơn một chút. Ta quyết định lấy kết quả chính là sự kết hợp của 3 model này. Chọn threshold là 0.33 vì nó cho F1-score là tốt nhất.

In [ ]:
pred_test_y = 0.33*pred_glove_test_y + 0.33*pred_wiki_news_test_y + 0.34*pred_paragram_test_y
pred_test_y = (pred_test_y>0.33).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)

Ta được output theo yêu cầu đề bài. prediction tương ứng với mỗi qid câu hỏi.

In [ ]:
out_df


Xem qua thử tầm 10 câu hỏi không chân thành xem máy dự đoán có chính xác không

In [ ]:
dem = 0;
for id,i in enumerate(pred_test_y):
    if i == 1:
        print(test_df['question_text'][id])
        dem = dem + 1
        if dem == 10:
            break

Có vẻ mô hình hoạt động khá ổn

# KẾT LUẬN VÀ ĐẶT VẤN ĐỀ

Như ta đã thấy:
- Việc sử dụng nhúng tốt hơn là việc không sử dụng
- 3 loại nhúng về cơ bản là cho hiệu suất giống như nhau, không thay đổi nhiều


Đặt vấn đề: Liệu ta có thể có hiệu suất tốt hơn nữa không?
Ta thấy việc hiệu suất phụ thuộc vào việc nhúng. Nghĩ lại một chút về lúc xây dựng mô hình GRU. Đối với các từ mà chưa xuất hiện trong file nhúng, ta đành phải lấy phân phối chuẩn của các từ đã xuất hiện. Vậy file nhúng chứa bao nhiêu % các từ đã biết rồi trong tổng số tất cả các từ trong các câu hỏi ?

Kiểm tra một chút về tỉ lệ phủ của các tool nhúng được cung cấp

In [ ]:
# Tổng hợp tất cả các câu hỏi lại để lấy ra tất cả các từ vựng trong đó
df = pd.concat([test_df, train_df])

In [ ]:
# load lại các file nhúng
embed_glove = load_embed(glove)
embed_wiki_news = load_embed(wiki_news)
embed_paragram = load_embed(paragram)

In [ ]:
# Xây dựng vocab chứa mọi từ vựng trong mọi câu hỏi và số lần xuất hiện của chúng
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
vocab = build_vocab(df['question_text'])


Hàm check độ phủ của các tool nhúng, độ phủ đối với vocab nghĩa là tool nhúng chứa bao nhiêu từ trong số lượng các từ khác nhau, độ phủ đối với all_text nghĩa là tool nhúng chứa bao nhiêu từ trong số tất cả các từ (tính cả số lượng của các từ trùng lặp)

In [ ]:
import operator 
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("Wiki news FastText : ")
oov_fasttext = check_coverage(vocab, embed_wiki_news)

Như ta thấy, Glove Embeddings chỉ phủ có 32% số lượng từ khác nhau trong các câu hỏi, cũng như là 88% trong tất cả các từ. Điều này cho thấy sự đa dạng của từ vựng trong Glove Embeddings là không nhiều. Paragram và Wiki news cũng vậy. Em nghĩ nếu ta tìm cách tăng được độ phủ của các tool nhúng, hiệu suất của bài toán cũng sẽ được tăng lên khá đáng kể!